In [1]:
import pandas as pd
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
import re
from  itertools import chain
from collections import Counter

# Load PNU table and table with eligible itens

In [2]:
df_pnu = pd.read_csv("DSMZ_bactnames.csv", sep="\t")
df_pnu["complete_name"] = df_pnu.apply(lambda x:'%s %s' % (x['GENUS'],x['SPECIES']),axis=1)

In [3]:
#function for obtaining the year of publication
def get_year(author):
    year_list = re.findall(r'\d\d\d\d', author)
    first_year = int(year_list[0])
    return(first_year)

df_pnu["YEAR"] = df_pnu["AUTHORS"].apply(get_year)


In [4]:
df_eligible = pd.read_csv("table_1_eligible.csv", index_col=[0]) #it reads table derived from medline file

# Validity of the extracted species names

In [5]:
text_mining_species = pd.read_csv("freq_species.csv", header=None)
text_mining_species.columns = ["species", "freq"]
text_mining_species

,species,freq
0,Flavobacterium aquaticum sp. nov,2
1,Chitinophaga terrae sp. nov,2
2,Deinococcus soli sp. nov,2
3,Sphingomonas ginsengisoli sp. nov,2
4,Aeromonas sharmana sp. nov,2
5,Aeromonas aquariorum sp. nov,2
6,Burkholderia humi sp. nov,2
7,Lysobacter humi sp. nov,2
8,Sphingomonas aeria sp. nov,2
9,Jeotgalibacillus soli sp. nov,2


In [6]:
def clean_name(name):
    list_name = name.split(" ")[0:2]
    clean_name = " ".join(list_name)
    return(clean_name)

In [7]:
text_mining_species["species_name"] = text_mining_species["species"].apply(clean_name)
text_mining_species

,species,freq,species_name
0,Flavobacterium aquaticum sp. nov,2,Flavobacterium aquaticum
1,Chitinophaga terrae sp. nov,2,Chitinophaga terrae
2,Deinococcus soli sp. nov,2,Deinococcus soli
3,Sphingomonas ginsengisoli sp. nov,2,Sphingomonas ginsengisoli
4,Aeromonas sharmana sp. nov,2,Aeromonas sharmana
5,Aeromonas aquariorum sp. nov,2,Aeromonas aquariorum
6,Burkholderia humi sp. nov,2,Burkholderia humi
7,Lysobacter humi sp. nov,2,Lysobacter humi
8,Sphingomonas aeria sp. nov,2,Sphingomonas aeria
9,Jeotgalibacillus soli sp. nov,2,Jeotgalibacillus soli


In [8]:
#Valid names available in DSMZ PNU table
valid_names = df_pnu[(pd.notna(df_pnu["SPECIES"])) & (pd.isna(df_pnu["SUBSPECIES"]))]
valid_names

,GENUS,SPECIES,SUBSPECIES,REFERENCE,STATUS,AUTHORS,REMARKS,RISK_GRP,TYPE_STRAINS,RECORD_NO,RECORD_LNK,complete_name,YEAR
0,Abditibacterium,utsteinense,NaN,68:3379,sp. nov. (VL),Tahon et al. 2018,NaN,NaN,"DSM 105287, LMG 29911, R-68213",797965,NaN,Abditibacterium utsteinense,2018
2,Abiotrophia,adiacens,NaN,45:798*,"comb. nov. (VP), homotypic syn.",(Bouvet et al. 1989) Kawamura et al. 1995,NaN,2,"ATCC 49175, CIP 103243, DSM 9848, GaD, CCUG 27...",772466,776611.0,Abiotrophia adiacens,1989
3,Abiotrophia,balaenopterae,NaN,49:503*,sp. nov. (VP),Lawson et al. 1999,NaN,NaN,"CCUG 37380, M1975/96/1, DSM 15827, ATCC 700813...",772467,776612.0,Abiotrophia balaenopterae,1999
4,Abiotrophia,defectiva,NaN,45:798*,comb. nov. (VP),(Bouvet et al. 1989) Kawamura et al. 1995,NaN,2,"ATCC 49176, CIP 103242, DSM 9849, SC10, CCUG 2...",772468,NaN,Abiotrophia defectiva,1989
5,Abiotrophia,elegans,NaN,49:1,sp. nov. (VL),Roggenkamp et al. 1999,NaN,2,"B1333, DSM 11693, ATCC 700633, CCUG 38949, CIP...",772469,776613.0,Abiotrophia elegans,1999
7,Abyssibacter,profundi,NaN,68:3424*,sp. nov. (VP),Zhou et al. 2018,NaN,NaN,"KCTC 52933, MCCC 1K03450, OUC007",797925,NaN,Abyssibacter profundi,2018
9,Abyssicoccus,albus,NaN,66:4299,sp. nov. (VL),Jiang et al. 2016,NaN,NaN,"YIM M12140, DSM 29158, CCTCC AB 2014213",794216,NaN,Abyssicoccus albus,2016
11,Abyssivirga,alkaniphila,NaN,66:1724*,sp. nov. (VP),Schouw et al. 2016,NaN,NaN,"L81, DSM 29592, JCM 30920",794021,NaN,Abyssivirga alkaniphila,2016
13,Acanthopleuribacter,pedis,NaN,58:2597*,sp. nov. (VP),Fukunaga et al. 2008,NaN,1,"FYK2218, KCTC 12899, NBRC 101209, DSM 28897",787601,NaN,Acanthopleuribacter pedis,2008
15,Acaricomes,phytoseiuli,NaN,56:465*,sp. nov. (VP),Pukall et al. 2006 emend. Nouioui et al. 2018,NaN,"1, n2","CCUG 49701, CSC, DSM 14247, JCM 14300",772478,NaN,Acaricomes phytoseiuli,2006


In [9]:
#Intersection between found species names (text mining) and valid names (DSMZ PNU)
pd.Series(list(set(text_mining_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                     Tolumonas osonensis
1                Planctobacterium marinum
2         Gallaecimonas pentaromativorans
3              Sphingomonas xinjiangensis
4                   Dyadobacter sediminis
5                     Bacillus paraflexus
6                       Microvirga lupini
7                    Undibacterium parvum
8                          Vibrio owensii
9                  Streptomyces catbensis
10                     Nocardia higoensis
11                        Borrelia sinica
12                 Nocardioides phosphati
13                  Clostridium tagluense
14            Bacteroides stercorirosoris
15                         Pantoea deleyi
16                   Halomonas malpeensis
17                Actinoallomurus acaciae
18               Chryseobacterium tructae
19           Flavobacterium denitrificans
20                Burkholderia xenovorans
21             Pelagicola litorisediminis
22                Methylobacterium oryzae
23                 Streptomyces ce

In [10]:
10079 / 10800 #93.32% valid

0.9332407407407407

# Valid names in Antonie van Leewenhoek

In [11]:
antonie = df_eligible[(df_eligible["Journal"] == "Antonie van Leeuwenhoek")]

antonie["species"] = antonie["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
antonie_species = pd.Series(list(chain.from_iterable(antonie["species"])))
antonie_freq_species = pd.DataFrame(antonie_species.value_counts()).reset_index()
antonie_freq_species.columns = ["species", "freq"]
antonie_freq_species["species_name"] = antonie_freq_species["species"].apply(clean_name)
antonie_freq_species

/home/fernando/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,species,freq,species_name
0,Paenibacillus beijingensis sp. nov,2,Paenibacillus beijingensis
1,Algisphaera agarilytica gen. nov,1,Algisphaera agarilytica
2,Isoptericola rhizophila sp. nov,1,Isoptericola rhizophila
3,Sphingomonas kyeonggiense sp. nov,1,Sphingomonas kyeonggiense
4,Sphingomonas ginsenosidivorax sp. nov,1,Sphingomonas ginsenosidivorax
5,Altererythrobacter terrae sp. nov,1,Altererythrobacter terrae
6,Streptomyces lonarensis sp. nov,1,Streptomyces lonarensis
7,Salinicoccus kekensis sp. nov,1,Salinicoccus kekensis
8,Micromonospora mangrovi sp. nov,1,Micromonospora mangrovi
9,Rhizobium rhizosphaerae sp. nov,1,Rhizobium rhizosphaerae


In [12]:
#Valid names in Antonie van Leewenhoek
pd.Series(list(set(antonie_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                   Nonomuraea candida
1              Thalassotalea sediminis
2             Salinicola peritrichatus
3                Alteromonas halophila
4             Enterovirga rhinocerotis
5         Kibdelosporangium banguiense
6           Streptococcus tangierensis
7              Janibacter alkaliphilus
8               Bowmanella dokdonensis
9             Kroppenstedtia sanguinis
10     Dactylosporangium solaniradicis
11               Micromonospora violae
12               Thalassococcus lentus
13               Roseomonas aeriglobus
14       Sinorhodobacter ferrireducens
15          Salinicoccus halitifaciens
16          Pelagicola litorisediminis
17          Planomonospora algeriensis
18           Salegentibacter echinorum
19                   Frankia inefficax
20            Granulosicoccus undariae
21            Streptomyces caldifontis
22         Pseudonocardia hispaniensis
23          Actinopolyspora saharensis
24        Deinococcus multiflagellatus
25                Hymenob

In [13]:
#Valid names in Antonie
527 / 709 #74.33%

0.7433004231311706

# Valid names in Archives of Microbiology


In [14]:
archives = df_eligible[(df_eligible["Journal"] == "Archives of microbiology")]

archives["species"] = archives["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
archives_species = pd.Series(list(chain.from_iterable(archives["species"])))
archives_freq_species = pd.DataFrame(archives_species.value_counts()).reset_index()
archives_freq_species.columns = ["species", "freq"]
archives_freq_species["species_name"] = archives_freq_species["species"].apply(clean_name)
archives_freq_species

/home/fernando/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,species,freq,species_name
0,Paludibacter jiangxiensis sp. nov,1,Paludibacter jiangxiensis
1,Jilinibacillus soli gen. nov,1,Jilinibacillus soli
2,Francisella piscicida sp. nov,1,Francisella piscicida
3,Bradyrhizobium sacchari sp. nov,1,Bradyrhizobium sacchari
4,Blautia glucerasei sp. nov,1,Blautia glucerasei
5,Lysinibacillus xyleni sp. nov,1,Lysinibacillus xyleni
6,Chromocurvus halotolerans gen. nov,1,Chromocurvus halotolerans
7,Lysobacter tongrenensis sp. nov,1,Lysobacter tongrenensis
8,Glycomyces xinjiangensis sp. nov,1,Glycomyces xinjiangensis
9,Chryseobacterium yeoncheonense sp. nov,1,Chryseobacterium yeoncheonense


In [15]:
#Valid names in Archives of Microbiology
pd.Series(list(set(archives_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                   Cellulosimicrobium marinum
1                            Roseospira marina
2                  Pseudomonas nitritireducens
3                       Arthrobacter enclensis
4                           Bacillus berkeleyi
5                     Arthrobacter ardleyensis
6                         Flavobacterium maris
7               Thiohalophilus thiocyanoxidans
8                       Roseospira navarrensis
9                           Dyadobacter flavus
10                       Cupriavidus plantarum
11                           Azoarcus buckelii
12                   Flavobacterium ahnfeltiae
13                    Bacterioplanes sanyensis
14                Novosphingobium endophyticum
15                    Kosmotoga arenicorallina
16                   Alkaliflexus imshenetskii
17                  Acinetobacter halotolerans
18                   Chromocurvus halotolerans
19                   Burkholderia panaciterrae
20                               Bacillus vini
21           

In [19]:
77 / 152 #50.66% valid

0.506578947368421

# Valid names in Current Microbiology

In [20]:
current = df_eligible[(df_eligible["Journal"] == "Current microbiology")]

current["species"] = current["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
current_species = pd.Series(list(chain.from_iterable(current["species"])))
current_freq_species = pd.DataFrame(current_species.value_counts()).reset_index()
current_freq_species.columns = ["species", "freq"]
current_freq_species["species_name"] = current_freq_species["species"].apply(clean_name)
current_freq_species

/home/fernando/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,species,freq,species_name
0,Flavobacterium chungangensis sp. nov,1,Flavobacterium chungangensis
1,Plastorhodobacter daqingensis gen. nov,1,Plastorhodobacter daqingensis
2,Citrobacter bitternis sp. nov,1,Citrobacter bitternis
3,Rhodanobacter xiangquanii sp. nov,1,Rhodanobacter xiangquanii
4,Mesoflavibacter profundi sp. nov,1,Mesoflavibacter profundi
5,Altererythrobacter estronivorus sp. nov,1,Altererythrobacter estronivorus
6,Caldicoprobacter algeriensis sp. nov,1,Caldicoprobacter algeriensis
7,Anaerococcus jeddahensis sp. nov,1,Anaerococcus jeddahensis
8,Marinobacter piscensis sp. nov,1,Marinobacter piscensis
9,Spirosoma gilvum sp. nov,1,Spirosoma gilvum


In [21]:
#Valid names in Current Microbiology
pd.Series(list(set(current_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0             Sphingomonas qilianensis
1                  Hymenobacter terrae
2            Pseudomonas benzenivorans
3            Gabonibacter massiliensis
4              Pseudomonas saponiphila
5                       Gramella lutea
6              Seohaeicola nanhaiensis
7             Mycobacterium hippocampi
8                     Lysobacter maris
9               Desulfovibrio legallis
10            Lutimaribacter litoralis
11            Polaribacter butkevichii
12           Actinomadura hankyongense
13             Aestuarium zhoushanense
14            Klebsiella michiganensis
15           Wenyingzhuangia aestuarii
16                Nocardioides dilutes
17       Flavobacterium qiangtangensis
18                      Spirosoma humi
19          Brevundimonas mongoliensis
20              Virgibacillus natechei
21    Pseudomonas seleniipraecipitatus
22          Marinobacterium marisflavi
23                   Enterobacter soli
24                      Cohnella ferri
25            Rhizobium s

In [22]:
70 / 161 #43.48% are valid in Current Microbiology

0.43478260869565216

# Valid names in IJSEM

In [24]:
ijsem = df_eligible[(df_eligible["Journal"] == "International journal of systematic and evolutionary microbiology")]

ijsem["species"] = ijsem["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
ijsem_species = pd.Series(list(chain.from_iterable(ijsem["species"])))
ijsem_freq_species = pd.DataFrame(ijsem_species.value_counts()).reset_index()
ijsem_freq_species.columns = ["species", "freq"]
ijsem_freq_species["species_name"] = ijsem_freq_species["species"].apply(clean_name)
ijsem_freq_species

/home/fernando/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,species,freq,species_name
0,Glaciimonas alpina sp. nov,2,Glaciimonas alpina
1,Pseudomonas furukawaii sp. nov,2,Pseudomonas furukawaii
2,Ochrobactrum pituitosum sp. nov,1,Ochrobactrum pituitosum
3,Longispora albida gen. nov,1,Longispora albida
4,Flavobacterium lindanitolerans sp. nov,1,Flavobacterium lindanitolerans
5,Corynebacterium crudilactis sp. nov,1,Corynebacterium crudilactis
6,Aurantimicrobium minutum gen. nov,1,Aurantimicrobium minutum
7,Sphingobacterium hotanense sp. nov,1,Sphingobacterium hotanense
8,Arthrobacter defluvii sp. nov,1,Arthrobacter defluvii
9,Larkinella knui sp. nov,1,Larkinella knui


In [25]:
#Valid names in IJSEM
pd.Series(list(set(ijsem_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                    Tolumonas osonensis
1               Planctobacterium marinum
2        Gallaecimonas pentaromativorans
3             Sphingomonas xinjiangensis
4                  Dyadobacter sediminis
5                    Bacillus paraflexus
6                      Microvirga lupini
7                   Undibacterium parvum
8                     Nocardia higoensis
9                 Streptomyces catbensis
10                       Borrelia sinica
11                 Clostridium tagluense
12                Nocardioides phosphati
13           Bacteroides stercorirosoris
14                        Pantoea deleyi
15               Actinoallomurus acaciae
16                  Halomonas malpeensis
17          Flavobacterium denitrificans
18               Burkholderia xenovorans
19               Methylobacterium oryzae
20                Streptomyces cerasinus
21                  Caldilinea aerophila
22                    Methylomonas lenta
23                Maribacter antarcticus
24              

In [27]:
8593 / 8736 #98.36% are valid

0.9836309523809523

# Valid names in Journal_of_microbiology_(Seoul,_Korea)


In [28]:
jom = df_eligible[(df_eligible["Journal"] == "Journal of microbiology (Seoul, Korea)")]

jom["species"] = jom["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
jom_species = pd.Series(list(chain.from_iterable(jom["species"])))
jom_freq_species = pd.DataFrame(jom_species.value_counts()).reset_index()
jom_freq_species.columns = ["species", "freq"]
jom_freq_species["species_name"] = jom_freq_species["species"].apply(clean_name)
jom_freq_species

/home/fernando/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,species,freq,species_name
0,Hymenobacter daeguensis sp. nov,1,Hymenobacter daeguensis
1,Niabella ginsenosidivorans sp. nov,1,Niabella ginsenosidivorans
2,Calculibacillus koreensis gen. nov,1,Calculibacillus koreensis
3,Devosia lucknowensis sp. nov,1,Devosia lucknowensis
4,Flavisolibacter swuensis sp. nov,1,Flavisolibacter swuensis
5,Dasania marina gen. nov,1,Dasania marina
6,Deinococcus rubellus sp. nov,1,Deinococcus rubellus
7,Ensifer collicola sp. nov,1,Ensifer collicola
8,Citrimicrobium luteum gen. nov,1,Citrimicrobium luteum
9,Achromobacter panacis sp. nov,1,Achromobacter panacis


In [29]:
#Valid names in Journal of microbiology
pd.Series(list(set(jom_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                  Roseovarius tibetensis
1                       Caenimonas terrae
2                      Acinetobacter apis
3                     Cohnella suwonensis
4                 Spirosoma montaniterrae
5                 Panacagrimonas perspica
6              Niabella ginsenosidivorans
7                        Limnobacter humi
8                  Paenibacillus pinesoli
9              Lysinibacillus tabacifolii
10                Flavobacterium koreense
11                      Chujaibacter soli
12            Rummeliibacillus suwonensis
13                     Spirosoma pulveris
14              Winogradskyella jejuensis
15                   Luteimonas aestuarii
16               Flavobacterium aquaticum
17    Diaminobutyricibacter tongyongensis
18               Flavobacterium jejuensis
19               Aestuariibaculum marinum
20                Hymenobacter daeguensis
21                   Halomonas tibetensis
22                  Antarcticimonas flava
23                   Rhodopirellul

In [30]:
92 / 172 #53.49% are valid in Journal of Microbiology

0.5348837209302325

# Valid names in SAM

In [32]:
sam = df_eligible[(df_eligible["Journal"] == "Systematic and applied microbiology")]

sam["species"] = sam["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
sam_species = pd.Series(list(chain.from_iterable(sam["species"])))
sam_freq_species = pd.DataFrame(sam_species.value_counts()).reset_index()
sam_freq_species.columns = ["species", "freq"]
sam_freq_species["species_name"] = sam_freq_species["species"].apply(clean_name)
sam_freq_species

/home/fernando/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,species,freq,species_name
0,Microbacterium endophyticum sp. nov,1,Microbacterium endophyticum
1,Vibrio barjaei sp. nov,1,Vibrio barjaei
2,Nonlabens agnitus sp. nov,1,Nonlabens agnitus
3,Rubrobacter calidifluminis sp. nov,1,Rubrobacter calidifluminis
4,Hymenobacter perfusus sp. nov,1,Hymenobacter perfusus
5,Porphyromonas pogonae sp. nov,1,Porphyromonas pogonae
6,Breznakia blatticola gen. nov,1,Breznakia blatticola
7,Hymenobacter metalli sp. nov,1,Hymenobacter metalli
8,Chimaereicella alkaliphila gen. nov,1,Chimaereicella alkaliphila
9,Alkanindiges hongkongensis sp. nov,1,Alkanindiges hongkongensis


In [34]:
#Valid names in SAM
pd.Series(list(set(sam_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                     Balneomonas flocculans
1            Pseudomonas laurylsulfatovorans
2                       Varibaculum anthropi
3                  Actinomyces hongkongensis
4                        Kocuria uropygialis
5                        Vibrio tasmaniensis
6                 Alkanindiges hongkongensis
7                          Meiothermus rufus
8               Micromonospora viridifaciens
9                         Orrella dioscoreae
10                       Clostridium bolteae
11                       Aeromonas piscicola
12                   Pseudomonas aestusnigri
13                  Endozoicomonas elysicola
14                         Devosia neptuniae
15                         Gilliamella bombi
16                 Clostridium saccharogumia
17                    Achromobacter animicus
18                      Chlamydia gallinacea
19                Breoghania corrubedonensis
20                Teichococcus ludipueritiae
21                Thermomonas hydrothermalis
22        

In [35]:
325 / 369 #88.08% are valid in SAM

0.8807588075880759

# Proportion of names validated between 2001 and 2018 that were contemplated in the study

In [16]:
#Names validated between 2001 and 2018
valid_names_XXI = df_pnu[(pd.notna(df_pnu["SPECIES"])) & (pd.isna(df_pnu["SUBSPECIES"])) & (df_pnu["YEAR"] > 2000)
                         & (df_pnu["YEAR"] < 2019)]
valid_names_XXI["complete_name"]

0          Abditibacterium utsteinense
7                Abyssibacter profundi
9                   Abyssicoccus albus
11             Abyssivirga alkaniphila
13           Acanthopleuribacter pedis
15              Acaricomes phytoseiuli
18       Acetanaerobacterium elongatum
20                 Acetatifactor muris
30           Acetoanaerobium pronyense
39              Acetobacter cerevisiae
40           Acetobacter cibinongensis
44                 Acetobacter fabarum
45               Acetobacter farinalis
46               Acetobacter ghanensis
48           Acetobacter indonesiensis
50                 Acetobacter lambici
53                 Acetobacter malorum
55                   Acetobacter musti
56         Acetobacter nitrogenifigens
58                    Acetobacter oeni
59             Acetobacter okinawensis
60              Acetobacter orientalis
63                 Acetobacter papayae
71                 Acetobacter persici
72                Acetobacter persicus
74            Acetobacter

In [17]:
#Intersection between found species names (text mining) and names validated in XXI century (DSMZ PNU)
pd.Series(list(set(text_mining_species["species_name"]).intersection(set(valid_names_XXI["complete_name"]))))

0                    Tolumonas osonensis
1               Planctobacterium marinum
2        Gallaecimonas pentaromativorans
3             Sphingomonas xinjiangensis
4                  Dyadobacter sediminis
5                    Bacillus paraflexus
6                      Microvirga lupini
7                   Undibacterium parvum
8                         Vibrio owensii
9                 Streptomyces catbensis
10                    Nocardia higoensis
11                       Borrelia sinica
12                Nocardioides phosphati
13                 Clostridium tagluense
14           Bacteroides stercorirosoris
15                        Pantoea deleyi
16                  Halomonas malpeensis
17               Actinoallomurus acaciae
18              Chryseobacterium tructae
19          Flavobacterium denitrificans
20               Burkholderia xenovorans
21            Pelagicola litorisediminis
22               Methylobacterium oryzae
23                Streptomyces cerasinus
24              

In [18]:
9962 / 12006 #82.98 % of the names validated between 2001 and 2018 were contemplated in the study

0.829751790771281